# 데이터 불러오기

In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
import re

In [3]:
barcode = pd.read_csv('/content/gdrive/MyDrive/cp1/barcode_full.csv')
barcode.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (4,6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Unnamed: 0,CLSBIZ_DT,SITE_ADDR,PRDLST_REPORT_NO,PRMS_DT,PRDLST_NM,BAR_CD,POG_DAYCNT,PRDLST_DCNM,BSSH_NM,END_DT,INDUTY_NM
0,0,NaN,충청북도 진천군 이월면 궁동길 51-24,19930443028350,20070119,면사랑 구수한 메밀국수,8802039211424,24개월 실온,건면,(주)면사랑,NaN,식품제조가공업
1,1,NaN,경기도 광주시 독고개길 156,1987036804531,20000110,종합캔디,8801696003397,제조일로부터18개월,캔디류,(주)한일제과,NaN,식품제조가공업
2,2,NaN,부산광역시 동구 충장대로 293(좌천동),19950129282236,20090724,좋은상품 유기농 부침가루,8805522061018,제조일로부터 12개월까지,기타가공품,사조동아원(주) 부산공장,20200217.0,식품제조가공업
3,3,NaN,경기도 동두천시 평화로2862번길 15(상봉암동),1997031009828,19971009,쵸코카라멜,8801649120355,제조일로부터 12개월,캔디류,한양제과주식회사,20161124.0,식품제조가공업
4,4,NaN,경상북도 영주시 소백로1916번길 34,201205540864,20120420,풍기홍삼愛홍삼정골드,8809359120061,제조일로부터 2년까지(실온),액상차,풍기홍삼애제조원,NaN,식품제조가공업


# Feature Engineering, 결측치 처리

In [4]:
# 결측치 확인, 삭제

barcode.isnull().sum()

Unnamed: 0               0
CLSBIZ_DT           171814
SITE_ADDR               30
PRDLST_REPORT_NO         0
PRMS_DT                  0
PRDLST_NM               34
BAR_CD                   0
POG_DAYCNT            5075
PRDLST_DCNM            211
BSSH_NM                 21
END_DT              135566
INDUTY_NM                4
dtype: int64

In [5]:
# 생산중단 상품 개수
barcode['END_DT'].notnull().sum()

# end_dt(생산중단 날짜)에 값이 있으면 생산중단제품이라 생각하고 삭제
barcode = barcode[barcode['END_DT'].isnull()]

In [6]:
# 바코드 중복값 갯수 확인
barcode['BAR_CD'].duplicated().sum()

# 바코드 중복값 삭제
barcode = barcode.drop_duplicates('BAR_CD')

In [7]:
# 사용할 column 선택
barcode = barcode[['PRDLST_NM','BAR_CD','POG_DAYCNT','PRDLST_DCNM','BSSH_NM']]

# column 명 변경
barcode.columns = ['제품명','바코드','유통기한','소분류','제조사']

In [8]:
# 결측치 삭제
barcode.dropna(how='any', inplace=True)

In [9]:
# 바코드 오타 제거, 데이터타입 변환 -> float)

def to_num(data):

    result = re.sub('[^0-9]', '', str(data))

    return result

barcode['바코드'] = barcode['바코드'].apply(to_num)
barcode['바코드'] = barcode['바코드'].astype('float')

In [10]:
print(barcode.shape)
barcode.head()

(121979, 5)


,제품명,바코드,유통기한,소분류,제조사
0,면사랑 구수한 메밀국수,8.802039e+12,24개월 실온,건면,(주)면사랑
1,종합캔디,8.801696e+12,제조일로부터18개월,캔디류,(주)한일제과
4,풍기홍삼愛홍삼정골드,8.809359e+12,제조일로부터 2년까지(실온),액상차,풍기홍삼애제조원
6,하이바나나,8.801074e+12,제조일로부터 14일(0~10도씨),가공유,삼양식품(주)유가공공장
9,고구마전분맛,8.803612e+12,24개월,전분가공품,전원식품


1.냉장고와 가장 관련 없을 거 같은 값 제거 ex) 히알루론산, 엽산 등등 영양제 같은거랑 건어포 당류가공품 같은 유통기한이 엄청 긴 것들 제거

2. 남은 것 중에 분류값마다 데이터 개수 확인하고 실제로 사용할만 한 것들 추가하기 ex)김치, 햄, 우유, 생햄, 모조치즈, 어육살, 어묵

# 소분류 설정

In [11]:
# 최종적으로 사용할 소분류 데이터

words = ['양념육','김치','어묵','소시지','햄','두부','자연치즈','우유','묵류','베이컨류','생햄','유당분해우유','혼합소시지','돼지고기','닭고기']

barcode = barcode[barcode['소분류'].isin(words)]

print(barcode['소분류'].shape)
barcode.head()

(9871,)


,제품명,바코드,유통기한,소분류,제조사
30,백김치,8.809217e+12,품질유지기한50일,김치,도미솔식품
41,유기농백김치,8.809335e+12,제조일로부터 60일,김치,농업회사법인주식회사담채원
62,포기김치,8.809358e+12,제조일로부터 90일(냉장보관),김치,주식회사 진미푸드지점
69,후레쉬밀크,8.801115e+12,냉장보관(0℃ ~10℃) : 제조일로부터 12일까지,우유,서울우유협동조합 안산공장
70,명품돈.삽겹바베큐,8.809354e+12,"제조일로부터 냉장(-2~5도) 45일, 냉동(-18도이하) 2년",햄,(주)산호푸드시스템


In [12]:
# 소분류 이름 변경

barcode.loc[barcode['소분류'] == '양념육', '소분류'] = '육류'
barcode.loc[barcode['소분류'] == '혼합소시지', '소분류'] = '소시지'
barcode.loc[barcode['소분류'] == '자연치즈', '소분류'] = '치즈'
barcode.loc[barcode['소분류'] == '생햄', '소분류'] = '햄'
barcode.loc[barcode['소분류'] == '유당분해우유', '소분류'] = '우유'
barcode.loc[barcode['소분류'] == '베이컨류', '소분류'] = '베이컨'
barcode.loc[barcode['소분류'] == '묵류', '소분류'] = '묵'

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [13]:
barcode['소분류'].value_counts()

육류      3525
김치      1649
어묵      1120
소시지      949
햄        828
두부       520
치즈       424
우유       396
묵        295
베이컨      155
돼지고기       6
닭고기        4
Name: 소분류, dtype: int64

# 유통기한 정수화

In [14]:
# 유통기한 일수와 관련없는 숫자 데이터 삭제
stop = ['0~10도씨', '영하18도', '0℃', '10℃', '-2~5도', '-18도',
'-18℃', '0~10℃', '-18℃이하', '-2℃', '5℃', '-18도씨', '-2 - 10℃', '-2 ℃~10℃',
'-2℃~5℃', '0~5도', '0-17도씨', '냉동-18', '400,800,920g', '14g', '-2～10℃',
'0～10℃', '1-35도', '14g', '-2도', '10도', '-2~10℃', '0~10도', '-2 ℃~5℃', '4도'
'1~35도씨', '0-10도', '-5도씨', '-2~5℃', '-2~-5℃', '0 ~ 10℃', '-2~10℃', 
'-2~5도', '-2~5도씨', '0-10도', '-1~3도씨', '0-5℃', '15~25℃', '1℃~35℃',
'1-35도', '-2도씨', '2005.05.04', '1)', '400g, 800g', '2)', '14g', '1-10℃', '18이하',
'50g', '300g', '500g', '800g', '1kg', '3.2kg', '15℃', '25℃', '400, 800, 920g', '14g',
'1-35도', '350mL', 'PET', 'CAN', '1.5L', '50g', '10g', '40g', '1월~6월', '7월~8월',
'9월~12월', '1~6월', '7~8월', '9~12월', '400, 800, 920g', '14g', '400g', '800g', '920g',
'13g', '0℃', '10℃', '-2~5도', '-18도', '18℃', '(-2', '18℃', '(0~', '0~10℃', '-2 - 10℃', '0~5도',
'-2～10℃', '-18℃', '0~', '(0', '-2℃', '5℃', '0~10도', '(4도시', '-2℃', '0-10도', '-2~-5℃',
'제조일', '(-1~3도씨)', '0-5℃', '-2도', '~5도', '10도', '(-18', '-2～5℃', '0~10도', '18도', 
'0-10℃', '-2~10도씨', '-2℃', '-2~5도', '2℃', '0-10도', '-2~10˚)', '0~10℃', '-2~10도', 
'-2˚~10˚C', '-2∼5℃', '0℃', '-2 도', '-2℃', '-2 ~ 10 ℃', '-2°', '10°', '1~35℃', '2˚', '(월,년)',
'24,48,', '-1도', '18도', '-2~10℃', '1)', '2)', '~1', '1이하', '℃~1', '1에서', '1 냉장', '7~',
'(~1', '~1냉장', '1에서', '(10˚C)', '10 ℃)', '7~', '1에서', '(10˚)', '0- 냉장', '1이하', '˚~10˚C',
'~ 10 ℃)', '˚~10˚C', '[1', '냉장10', '1 이하', '∼10', '1에서', '냉장1~', '(10˚c)', '10 ℃', '1 보관',
'12월까지', '(~1이하)', '℃~1', '0 ~1에서', '0 ～ 1 에서', '~1도', '6~9월', '15월', '1에서',
'℃~1', '4-10월', '11-3월', '4월~10월', '1℃-3', '11월~3월', '(1,냉장보관)', '영하2도',
'-2 ℃~1보관', '4-10월=24시간, 11-3월=48시간', '10 이하온도', '0-1 로부터', ' (~10이하) ',
'-18', '(10 ℃', '10c)', '(2~', '2도', '4월~10월', '11월~3월', '~1 이하', '11~3월' ,'4~10월', '()3',
'(1', '-1에서', '-2 ℃ ', '10 ℃', '-18 ℃', '냉장~1', '/~1이하', '2이하', '-2 ~', '0 ~ 1에서', 
'~ 10 ℃', '(10˚c)', '∼10 ℃)', '10이하' ,'-2 ℃', '10 ℃', '4-10월', '11-3월', '～1이하', '5도',
'0 ~1에서', '(1 냉장보관)', '냉장(10˚C)', "-10')" ,'~1보관시', '(냉장)1', '~1사이', '(1 보관)',
'~ 10 ℃', '~ 1 냉장', '로부터10 (이하)', '0도', '0～1에서', '0-1', '( -2 ~ 1 )', ' ( - 18 ℃ )',
'10 ℃이하', '( 0 ~ )', '냉장~1', '(1 냉장보관)', '~ 10 ℃)', '(1 이하)', '~1냉장보관-', '(1, 냉장보관)',
'6-9월', '10-5월', '( ~ 5도)', '( 5도이하보관)', '[1 냉장보관]', '0도', '1. 냉장제품:', '( 0 ~ 10 ℃에서 냉장보관 )',
'(~1 )', '~1이하보관시)', '(10 ℃ 냉장보관)', '(10 ℃ 냉장보관)', '( 10 ℃ 냉장보관 )', '0-1 냉장',
'(1의', '~1에서', '-2에서']


def to_date(data):
    for word in stop:
        if word in data:
            data = data.replace(word, '')

    return data

barcode['유통기한'] = barcode['유통기한'].apply(to_date)

In [15]:
# 정규표현식 작업을 위해 단위를 알파벳으로 바꿔줌

def upper_alphabet(data):
    if '년' in data:
        data = data.replace('년','Y')
    if '개월' in data:
        data = data.replace('개월','M')
    if '월' in data:
        data = data.replace('월', 'M')
    if '주' in data:
        data = data.replace('주', 'W')
    if '일' in data:
        data = data.replace('일','D')
    if '시간' in data:
        data = data.replace('시간','H')
    
    return data

barcode['유통기한'] = barcode['유통기한'].apply(upper_alphabet)

In [16]:
# 숫자와 알파벳대문자를 제외한 문자 삭제
import re

def clean(data):
    
    data = re.sub('[^A-Z0-9]','', data)

    return data

barcode['유통기한'] = barcode['유통기한'].apply(clean)

In [17]:
# 빈 값을 가진 행 삭제

barcode['유통기한'].replace('', np.nan, inplace=True)
barcode.dropna(how='any', inplace=True)

In [18]:
def contour(data):
        
    if 'Y' in data:
        data = data.replace('Y','Y/')
        data = data.replace(' ','')
    if 'M' in data:
        data = data.replace('M','M/')
        data = data.replace(' ','')
    if 'W' in data:
        data = data.replace('W', 'W/')
        data = data.replace(' ','')
    if 'D' in data:
        data = data.replace('D','D/')
        data = data.replace(' ','')
    if 'H' in data:
        data = data.replace('H','H/')
        data = data.replace(' ','')
    else:
        data = data.replace(' ','')
    
    return data

barcode['유통기한'] = barcode['유통기한'].apply(contour)

In [19]:
# / 를 중심으로 구분해주기

def classify(data):
    data = data.split('/')
    
    return data

barcode['유통기한'] = barcode['유통기한'].apply(classify)

In [20]:
# 유통기한 단위 '일'로 통일시키고 최솟값 반환하기

def expiry_date(data):
    res = []
    for i, val in enumerate(data):
        try:
            only_num = int(''.join(filter(str.isdigit, val)))

            if 48 <= ord(val[-1]) <= 57:
                flag_num = 1
            elif val[-1] == 'Y':
                flag_num = 365
            elif val[-1] == 'M':
                flag_num = 30
            elif val[-1] == 'W':
                flag_num = 7
            elif val[-1] == 'D':
                flag_num = 1
            elif val[-1] == 'H':
                flag_num = 1/24

        except ValueError as e:
            res.append(np.nan)
        else:
            res.append(only_num * flag_num)
        
    return min(res)

barcode['유통기한'] = barcode['유통기한'].apply(expiry_date)

In [21]:
# 정수형으로 변경
barcode.dropna(how='any', inplace=True)
barcode['유통기한'] = barcode['유통기한'].astype(int)

In [22]:
# index 초기화
barcode = barcode.reset_index()
barcode = barcode.drop('index', axis=1)

In [25]:
def labeling (data): # 소비기한-유통기한 라벨링

  data.loc[barcode['소분류'] == '육류', '소비기한-유통기한'] = 180
  data.loc[barcode['소분류'] == '김치', '소비기한-유통기한'] = 180
  data.loc[barcode['소분류'] == '어묵', '소비기한-유통기한'] = 3
  data.loc[barcode['소분류'] == '소시지', '소비기한-유통기한'] = 7
  data.loc[barcode['소분류'] == '햄', '소비기한-유통기한'] = 7
  data.loc[barcode['소분류'] == '두부', '소비기한-유통기한'] = 90
  data.loc[barcode['소분류'] == '치즈', '소비기한-유통기한'] = 70 
  data.loc[barcode['소분류'] == '우유', '소비기한-유통기한'] = 45

  data['소비기한-유통기한'] = data['소비기한-유통기한'].fillna(0)

  return data

barcode = labeling(barcode)

In [26]:
print(barcode.shape)
barcode.head()

(9466, 6)


,제품명,바코드,유통기한,소분류,제조사,소비기한-유통기한
0,백김치,8.809217e+12,50,김치,도미솔식품,180.0
1,유기농백김치,8.809335e+12,60,김치,농업회사법인주식회사담채원,180.0
2,포기김치,8.809358e+12,90,김치,주식회사 진미푸드지점,180.0
3,후레쉬밀크,8.801115e+12,12,우유,서울우유협동조합 안산공장,45.0
4,명품돈.삽겹바베큐,8.809354e+12,45,햄,(주)산호푸드시스템,7.0


# 전처리 완료한 파일 내보내기

In [24]:
# barcode_cp1.csv로 내보내기

barcode.to_csv('cp1_barcode.csv', index=False, encoding='CP949')